In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
# Example usage
from data_gen import generate_fashion_data_with_brand, inject_anomalies_by_date

start_date = '2023-01-01'
end_date = '2023-12-30'
df = generate_fashion_data_with_brand(start_date, end_date)

In [3]:
df['Category'] = df['MERCHANDISE_HIERARCHY'].str.split('.').str[0]
df['SubCategory'] = df['MERCHANDISE_HIERARCHY'].str.split('.').str[1]

In [4]:
anomaly_schedule = {
    '2023-01-10': ('ExcessiveDiscount', 0.6, 'PricingError', 'Apparel'),
    '2023-06-10': ('COGSOverstatement', 0.8, 'SupplierIssue', 'Footwear'),
    '2023-09-10': ('FulfillmentSpike', -2, 'LogisticsIssue', 'Beauty'),
    '2023-12-10': ('ReturnSurge', -6, 'QualityIssue', 'Accessories')
}

df_anomalous = inject_anomalies_by_date(df, anomaly_schedule)

In [5]:
# Convert 'ORDERDATE' to datetime objects if not already done
df_anomalous['ORDERDATE'] = pd.to_datetime(df_anomalous['ORDERDATE'])

# Define categorical variables for grouping
categorical_variables = [
    "Category", "SubCategory", "PROMO_CODE", "SALES_CHANNEL", "CUSTOMER_LOYALTY"
]

# Define numerical/fiscal variables for aggregation
numerical_variables = [
    "PRICEEACH", "UNIT_COST", "QUANTITYORDERED", "SALES", "DISCOUNT", "NET_SALES",
    "FULFILLMENT_COST", "MARKETING_COST", "RETURN_COST", "COST_OF_GOODS_SOLD",
    "SHIPPING_REVENUE", "PROFIT", "PROFIT_MARGIN", "IS_MARGIN_NEGATIVE"
]

# Define aggregation methods for numerical variables
aggregation_dict = {
    "PRICEEACH": "mean",  # Average price per unit within each group
    "UNIT_COST": "mean",  # Average cost per unit
    "QUANTITYORDERED": "sum",  # Total units sold within each group per day
    "SALES": "sum",  # Total sales revenue
    "DISCOUNT": "sum",  # Total discounts applied
    "NET_SALES": "sum",  # Total net sales
    "FULFILLMENT_COST": "sum",  # Total fulfillment costs
    "MARKETING_COST": "sum",  # Total marketing costs
    "RETURN_COST": "sum",  # Total return costs
    "COST_OF_GOODS_SOLD": "sum",  # Total cost of goods sold
    "SHIPPING_REVENUE": "sum",  # Total shipping revenue
    "PROFIT": "sum",  # Total profit
    "PROFIT_MARGIN": "mean",  # Average profit margin (weighted by NET_SALES if needed)
    "IS_MARGIN_NEGATIVE": "mean"  # Proportion of negative margins (0 or 1)
}

# Group by ORDERDATE and categorical variables, then aggregate numerical variables
df_agg = df_anomalous.groupby([pd.Grouper(key='ORDERDATE', freq='D')] + categorical_variables).agg(aggregation_dict).reset_index()

df_agg.head()

,ORDERDATE,Category,SubCategory,PROMO_CODE,SALES_CHANNEL,CUSTOMER_LOYALTY,PRICEEACH,UNIT_COST,QUANTITYORDERED,SALES,DISCOUNT,NET_SALES,FULFILLMENT_COST,MARKETING_COST,RETURN_COST,COST_OF_GOODS_SOLD,SHIPPING_REVENUE,PROFIT,PROFIT_MARGIN,IS_MARGIN_NEGATIVE
0,2023-01-01,Accessories,Handbags,FREE10,B2BCustomers:Corporate Client,New,425.01,246.02,2,850.02,97.92,752.10,9.90,1.99,22.56,492.04,0.0,225.61,30.00,0.0
1,2023-01-01,Accessories,Handbags,FREE10,B2BCustomers:Reseller,Loyal,180.36,96.19,2,360.72,23.09,337.63,9.90,1.83,10.13,192.38,0.0,123.39,36.55,0.0
2,2023-01-01,Accessories,Handbags,FREE10,OnlineStore:Mobile App,Loyal,392.92,235.64,2,785.84,45.26,740.58,11.88,1.64,22.22,471.28,0.0,233.56,31.54,0.0
3,2023-01-01,Accessories,Handbags,FREE10,RetailOutlet:Outlet Store,New,305.03,161.03,1,305.03,29.28,275.75,7.00,3.71,8.27,161.03,0.0,95.74,34.72,0.0
4,2023-01-01,Accessories,Handbags,NO_CODE,B2BCustomers:Corporate Client,New,232.01,137.94,1,232.01,0.00,232.01,7.70,2.13,6.96,137.94,0.0,77.28,33.31,0.0


In [6]:
# Assume df is your DataFrame loaded with the data
df_agg_enc = df_agg.copy()

le_category = LabelEncoder()
df_agg_enc['Category'] = le_category.fit_transform(df_agg_enc['Category']).astype(str)

le_subcategory = LabelEncoder()
df_agg_enc['SubCategory'] = le_subcategory.fit_transform(df_agg_enc['SubCategory']).astype(str)

le_promo = LabelEncoder()
df_agg_enc['PROMO_CODE'] = le_promo.fit_transform(df_agg_enc['PROMO_CODE']).astype(str)

le_sales_channel = LabelEncoder()
df_agg_enc['SALES_CHANNEL'] = le_sales_channel.fit_transform(df_agg_enc['SALES_CHANNEL']).astype(str)

# Ordinal Encoding for CUSTOMER_LOYALTY:
# Define the order explicitly (e.g., "New" comes before "Loyal")
ordinal_mapping = {"New": 0, "Loyal": 1}
df_agg_enc['CUSTOMER_LOYALTY'] = df_agg_enc['CUSTOMER_LOYALTY'].map(ordinal_mapping).astype(str)

In [13]:
dag_variables = [
    "PRICEEACH", "UNIT_COST", "SALES", "QUANTITYORDERED",
    "FULFILLMENT_COST", "COST_OF_GOODS_SOLD", "SHIPPING_REVENUE",
    "DISCOUNT", "NET_SALES", "PROMO_CODE", "SALES_CHANNEL",
    "CUSTOMER_LOYALTY", "Category", "MARKETING_COST", "RETURN_COST",
    "SubCategory", "PROFIT", "PROFIT_MARGIN"
]


edges = [
    # PRICEEACH influences UNIT_COST and SALES
    ("PRICEEACH", "UNIT_COST"),
    ("PRICEEACH", "SALES"),

    # QUANTITYORDERED influences SALES, FULFILLMENT_COST, COST_OF_GOODS_SOLD, and SHIPPING_REVENUE
    ("QUANTITYORDERED", "SALES"),
    ("QUANTITYORDERED", "FULFILLMENT_COST"),
    ("QUANTITYORDERED", "COST_OF_GOODS_SOLD"),
    ("QUANTITYORDERED", "SHIPPING_REVENUE"),

    # SALES influences DISCOUNT, NET_SALES, and SHIPPING_REVENUE
    ("SALES", "DISCOUNT"),
    ("SALES", "NET_SALES"),
    ("SALES", "SHIPPING_REVENUE"),

    # PROMO_CODE is used in the discount calculation
    ("PROMO_CODE", "DISCOUNT"),

    # SALES_CHANNEL factors into DISCOUNT and FULFILLMENT_COST calculations
    ("SALES_CHANNEL", "DISCOUNT"),
    ("SALES_CHANNEL", "FULFILLMENT_COST"),

    # CUSTOMER_LOYALTY factors into the discount
    ("CUSTOMER_LOYALTY", "DISCOUNT"),

    # MERCHANDISE_HIERARCHY (which defines the product category) influences DISCOUNT, MARKETING_COST, RETURN_COST, FULFILLMENT_COST, and PRODUCTCODE
    ("Category", "DISCOUNT"),
    ("Category", "MARKETING_COST"),
    ("Category", "RETURN_COST"),
    ("Category", "FULFILLMENT_COST"),


    ("SubCategory", "DISCOUNT"),
    ("SubCategory", "MARKETING_COST"),
    ("SubCategory", "RETURN_COST"),
    ("SubCategory", "FULFILLMENT_COST"),

    # UNIT_COST and QUANTITYORDERED combine to form COST_OF_GOODS_SOLD
    ("UNIT_COST", "COST_OF_GOODS_SOLD"),

    # DISCOUNT influences NET_SALES (net sales = sales - discount)
    ("DISCOUNT", "NET_SALES"),

    # NET_SALES is used to compute RETURN_COST, PROFIT, and later PROFIT_MARGIN
    ("NET_SALES", "RETURN_COST"),
    ("NET_SALES", "PROFIT"),
    ("NET_SALES", "PROFIT_MARGIN"),

    # The cost components feed into PROFIT
    ("FULFILLMENT_COST", "PROFIT"),
    ("MARKETING_COST", "PROFIT"),
    ("RETURN_COST", "PROFIT"),
    ("COST_OF_GOODS_SOLD", "PROFIT"),
    ("SHIPPING_REVENUE", "PROFIT"),

    # PROFIT drives PROFIT_MARGIN and IS_MARGIN_NEGATIVE
    ("PROFIT", "PROFIT_MARGIN"),
]

# --- Convert DAG to Dummy Adjacency Matrix ---
n_vars = len(dag_variables)
# Initialize an n_vars x n_vars matrix of zeros
adj_matrix = np.zeros((n_vars, n_vars), dtype=int)

# Map variable names to indices
var_to_idx = {var: idx for idx, var in enumerate(dag_variables)}
for src, tgt in edges:
    i = var_to_idx[src]
    j = var_to_idx[tgt]
    adj_matrix[i, j] = 1
# Convert to a DataFrame for later evaluation
true_matrix = pd.DataFrame(adj_matrix, columns=dag_variables, index=dag_variables)

# --- Split Data into Normal and Abnormal ---
# For example, use data before 2023-06-01 as training (normal) and data after as abnormal.
training_data = df_agg_enc[~df_agg_enc['ORDERDATE'].isin(['2023-01-10', '2023-06-10', '2023-09-10', '2023-12-10'])]
abnormal_data = df_agg_enc[df_agg_enc['ORDERDATE'].isin(['2023-01-10', '2023-06-10', '2023-09-10', '2023-12-10'])]

# Create DataFrames with only the DAG variables
training_data_df = training_data[dag_variables]
abnormal_data_df = abnormal_data[dag_variables]


C:\Users\Ahmed Dawood\AppData\Local\Temp\ipykernel_21084\3513785432.py:86: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  training_data = df_agg_enc[~df_agg_enc['ORDERDATE'].isin(['2023-01-10', '2023-06-10', '2023-09-10', '2023-12-10'])]
C:\Users\Ahmed Dawood\AppData\Local\Temp\ipykernel_21084\3513785432.py:87: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  abnormal_data = df_agg_enc[df_agg_enc['ORDERDATE'].isin(['2023-01-10', '2023-06-10', '2023-09-10', '2023-12-10'])]


In [23]:
from ht_update import HT, HTConfig
# Example: Load or define an adjacency matrix as a pandas DataFrame.
# For instance, assume a CSV file 'graph.csv' with an adjacency matrix.
config = HTConfig(graph=true_matrix, aggregator="max", root_cause_top_k=3, model_type='Xgboost') #  "LinearRegression", "CatBoost", "Xgboost", "LightGBM", "RandomForest".
# Create the HT instance.
ht_algo = HT(config)

# Train the regression models.
ht_algo.train(training_data_df)

# Find root causes from the abnormal data. Optionally, specify an anomalous metric.
results = ht_algo.find_root_causes(abnormal_data_df, anomalous_metrics="PROFIT_MARGIN", return_paths=True, adjustment=True)

# Display results.
results.to_dict()

{'root_cause_nodes': [('RETURN_COST', 644.0777125171726),
  ('NET_SALES', 458.0111273294927),
  ('FULFILLMENT_COST', 232.666261202223)],
 'root_cause_paths': {'RETURN_COST': ['RETURN_COST',
   'PROFIT',
   'PROFIT_MARGIN'],
  'NET_SALES': ['NET_SALES', 'PROFIT_MARGIN'],
  'FULFILLMENT_COST': ['FULFILLMENT_COST', 'PROFIT', 'PROFIT_MARGIN']}}

In [24]:
# --- Run Root Cause Analysis on Each Anomaly Date ---
# List the anomaly dates we injected
anomaly_dates = ['2023-01-10', '2023-06-10', '2023-09-10', '2023-12-10']

In [25]:
for date in anomaly_dates:
    print(f"\n===== Anomaly Date: {date} =====")
    # Filter the aggregated abnormal data by the specific anomaly date.
    # (df_agg_enc includes ORDERDATE; abnormal_data_df may not.)
    date_filtered = df_agg_enc[df_agg_enc['ORDERDATE'] == pd.to_datetime(date)]
    if date_filtered.empty:
        print(f"No data found for anomaly date {date}")
        continue

     # Restrict to the DAG variables (the metrics used in your causal graph)
    date_filtered_df = date_filtered[dag_variables]

    # HT-based Root Cause Analysis (using "PROFIT_MARGIN" as the indicator)
    print("\n-- HT Results --")
    ht_results = ht_algo.find_root_causes(date_filtered_df, "PROFIT_MARGIN", return_paths=True).to_list()
    for res in ht_results:
        print(res)


===== Anomaly Date: 2023-01-10 =====

-- HT Results --
{'root_cause': 'DISCOUNT', 'score': 99.18314414562556, 'paths': ['DISCOUNT', 'NET_SALES', 'PROFIT_MARGIN']}
{'root_cause': 'COST_OF_GOODS_SOLD', 'score': 17.726804224567598, 'paths': ['COST_OF_GOODS_SOLD', 'PROFIT', 'PROFIT_MARGIN']}
{'root_cause': 'SALES', 'score': 16.347653173763202, 'paths': ['SALES', 'NET_SALES', 'PROFIT_MARGIN']}

===== Anomaly Date: 2023-06-10 =====

-- HT Results --
{'root_cause': 'PROFIT', 'score': 45.94062976747211, 'paths': ['PROFIT', 'PROFIT_MARGIN']}
{'root_cause': 'COST_OF_GOODS_SOLD', 'score': 11.803688434623052, 'paths': ['COST_OF_GOODS_SOLD', 'PROFIT', 'PROFIT_MARGIN']}
{'root_cause': 'SALES', 'score': 10.009202752578158, 'paths': ['SALES', 'NET_SALES', 'PROFIT_MARGIN']}

===== Anomaly Date: 2023-09-10 =====

-- HT Results --
{'root_cause': 'FULFILLMENT_COST', 'score': 33.47281391512919, 'paths': ['FULFILLMENT_COST', 'PROFIT', 'PROFIT_MARGIN']}
{'root_cause': 'PROFIT_MARGIN', 'score': 31.7375869320